**[ABORTED] It was decided to use random dataset with more records and less geo/address information**

# Feature engineering 
### Preparing the data to the model

Author: Karina Condeixa

**[WIP]**

**Content**

1- open mock_dataset.

2- join toilette dataset to have its `address + postcode + lat + lng`.

3- measure distances between item and user looker.

4- add the column `distance_item_user`.

**Coments that came from moock_dataset_file:** 
- coordinates of user who post should became the ones for available item.
- we don't have info about item's type and category in the app, could be needed to parse posts' titles and description.
- UXs are still decuting and still don't havve a decision about having number of stars to a post and the classifications in conditions. I will adopt 1, 2 and 3 for conditions and add the number of starts. For condition, 1 is the best and 3 the worse. For stars, 5 is the best and 1 is the worse.

**[TODO]**

- add rank

- decide about using a fake dataset with 1500 records or
- a dataset with 330 records that match `address + postcode + lat + lng`

**Possible approaches:**
- item based (items'condition, number of stars).
- collaboractive: user iteraction based (likes, views).
- **-> hybrid**

In [4]:
import pandas as pd
# import numpy as np

In [5]:
mock_data_original = pd.read_csv('data/model_mock_data.csv', index_col=False)
mock_data_original.head()

,item_id,item_name-item_category-item,item_condition,item_lat_lng,item_postcode,item_status,userwhoposted_id,userwhopickedup_id,userwhochangeditemstatus_id,userwhochangeditemstatus_lat_lng,searched_item_name-searched_item_category-searched_item,searched_postcode,item_datetime_posted,item_datetimechangeditemstatus
0,485,furniture-rollcontainers,medium_conditionbad_condition,[<shapely.geometry.point.Point object at 0x7fb...,13409,not_available,246,188,922,[<shapely.geometry.point.Point object at 0x7fb...,miscelaneaous-bicycle,13347,2023-02-23,2023-02-25
1,584,miscelaneaous-ski_skates,good_condition,[<shapely.geometry.point.Point object at 0x7fb...,12349,not_available,900,256,652,[<shapely.geometry.point.Point object at 0x7fb...,furniture-cabinet,12103,2023-02-23,2023-02-23
2,953,appliance-washing_machine,good_condition,[<shapely.geometry.point.Point object at 0x7fb...,13465,not_available,178,326,512,[<shapely.geometry.point.Point object at 0x7fb...,clothes-woman_jacket,10783,2023-02-21,2023-02-21
3,126,clothes-child_clothes,medium_conditionbad_condition,[<shapely.geometry.point.Point object at 0x7fb...,13127,avaliable,697,730,81,[<shapely.geometry.point.Point object at 0x7fb...,clothes-woman_jacket,12623,2023-02-22,2023-02-24
4,739,clothes-woman_clothes,good_condition,[<shapely.geometry.point.Point object at 0x7fb...,13435,not_available,409,717,572,[<shapely.geometry.point.Point object at 0x7fb...,appliance-sandwich_maker,14195,2023-02-20,2023-02-22


In [6]:
# item_cols = ['item_id', 'item_postcode', 'item_lat', 'item_lng', 'item_category', 'item_name', 'item_condition']

data_cols = ['item_id','item_address', 'item_postcode', 'item_lat', 'item_lng', 'item_condition', 'n_stars', 'user_lat', 'user_lng', 'post_likes', 'distance_pubtransp']

# item_data = item_data[item_cols]


In [7]:
mock_data = mock_data_original.copy()

### Adding info from toiletes in the mock dataset (postcode + address + lat + lng)

In [9]:
toilette_data = pd.read_excel('data/berliner-toiletten-standorte.xlsx', skiprows=[0,1,2])
toilette_data.head()
# toilette_data['Street'].unique().size

,LavatoryID,Description,City,Street,Number,PostalCode,Country,Longitude,Latitude,isOperatedByWall,...,FID,neu vergebene FID,contract,modelTyp,Benutzungs-entgelt bis 8/2022,entgeltfrei Phase1 (8/22-2/23),entgeltfrei in Phase 2 (ab 2/23),neu in Phase 2 (ab 2/23),Price_new,Price_new.1
0,Wall_5339,Toilette Alexanderplatz,Berlin,Alexanderplatz 1 (10-22 Uhr),NaN,10178.0,Deutschland,"13,412954","52,521463",1,...,2,NaN,1,7,1.0,0.0,0.0,0.0,0.5,0.5
1,Wall_5348,Toilette Breite Str.,Berlin,"Spandauer Markt , Markt 5 (10-18 Uhr)",NaN,13597.0,Deutschland,"13,2050277777778","52,5368611111111",1,...,4,NaN,1,7,1.0,0.0,0.0,0.0,0.5,0.5
2,Wall_120224,"Berliner Toilette, Hohenzollerndamm",Berlin,Hohenzollerndamm (0-24 Uhr),NaN,14199.0,Deutschland,"13,2818898456133","52,4766788267014",1,...,6,NaN,1,1,1.0,1.0,1.0,0.0,0.0,0.0
3,Wall_120941,"Berliner Toilette, Rathaus Spandau, Altstädter...",Berlin,"Rathaus Spandau, Altstädter Ring (0-24 Uhr)",NaN,13597.0,Deutschland,"13,2004290600801","52,5354700822998",1,...,7,NaN,1,3,1.0,1.0,1.0,0.0,0.0,0.0
4,Wall_120494,"Berliner Toilette, Schloßstr.",Berlin,Schloßstr. ggü. 33-35 (0-24 Uhr),NaN,12163.0,Deutschland,"13,3214325154","52,457269514",1,...,9,NaN,1,3,1.0,1.0,1.0,0.0,0.0,0.0


In [10]:
toilette_data = toilette_data.dropna(subset = ['PostalCode'])

In [11]:
toilette_data['PostalCode'].unique()

array([10178., 13597., 14199., 12163., 12355., 10789., 13507., 10707.,
       10557., 10243., 13353., 10627., 10967., 14165., 10317., 10249.,
       13156., 12161., 12099., 10709., 10551., 12681., 12307., 12353.,
       10435., 12101., 13465., 10318., 14089., 12209., 13088., 13187.,
       10997., 13595., 12107., 13627., 10717., 10437., 12247., 13585.,
       14059., 12587., 10825., 10785., 12679., 12683., 12527., 12619.,
       14163., 13405., 14195., 10781., 12487., 13437., 10961., 10999.,
       13189., 10439., 10365., 10245., 12109., 10969., 14193., 10715.,
       13355., 13059., 10719., 12437., 10409., 12055., 14197., 10827.,
       13349., 10247., 12157., 13125., 13359., 12435., 10783., 10179.,
       12689., 13053., 10555., 12277., 12621., 12167., 13086., 12053.,
       13127., 12049., 12309., 12623., 12057., 12627., 12169., 12103.,
       10117., 12105., 12459., 12051., 12489., 14053., 10367., 12207.,
       12589., 12524., 14052., 10965., 10369., 13347., 10559., 12359.,
      

In [12]:
# toilette_data['Latitude'].isna().sum()
toilette_data['Longitude'].isna().sum()

0

In [13]:
# toilette_data['Latitude'] = toilette_data['Latitude'].values.astype('str')
# toilette_data['Longitude'] = toilette_data['Longitude'].values.astype('str')

# latitude = toilette_data['Latitude'] 
# longitude = toilette_data['Longitude'] 


toilette_data['Latitude'] = toilette_data['Latitude'].replace(',','.',regex=True)
toilette_data['Longitude']  = toilette_data['Longitude'].replace(',','.',regex=True) 
toilette_data['PostalCode'] = toilette_data['PostalCode'].astype(int)

toilette_data['Latitude'] = toilette_data['Latitude'].astype(float)
toilette_data['Longitude'] = toilette_data['Longitude'].astype(float)

toilette_data.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356 entries, 0 to 447
Data columns (total 27 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   LavatoryID                        356 non-null    object 
 1   Description                       356 non-null    object 
 2   City                              356 non-null    object 
 3   Street                            356 non-null    object 
 4   Number                            0 non-null      float64
 5   PostalCode                        356 non-null    int64  
 6   Country                           356 non-null    object 
 7   Longitude                         356 non-null    float64
 8   Latitude                          356 non-null    float64
 9   isOperatedByWall                  356 non-null    int64  
 10  isHandicappedAccessible           356 non-null    int64  
 11  Price                             356 non-null    float64
 12  canBePay

In [14]:
mock_data = mock_data.iloc[:356,:]
mock_data.head()

# Report_Card.iloc[6:12,2:]

,item_id,item_name-item_category-item,item_condition,item_lat_lng,item_postcode,item_status,userwhoposted_id,userwhopickedup_id,userwhochangeditemstatus_id,userwhochangeditemstatus_lat_lng,searched_item_name-searched_item_category-searched_item,searched_postcode,item_datetime_posted,item_datetimechangeditemstatus
0,485,furniture-rollcontainers,medium_conditionbad_condition,[<shapely.geometry.point.Point object at 0x7fb...,13409,not_available,246,188,922,[<shapely.geometry.point.Point object at 0x7fb...,miscelaneaous-bicycle,13347,2023-02-23,2023-02-25
1,584,miscelaneaous-ski_skates,good_condition,[<shapely.geometry.point.Point object at 0x7fb...,12349,not_available,900,256,652,[<shapely.geometry.point.Point object at 0x7fb...,furniture-cabinet,12103,2023-02-23,2023-02-23
2,953,appliance-washing_machine,good_condition,[<shapely.geometry.point.Point object at 0x7fb...,13465,not_available,178,326,512,[<shapely.geometry.point.Point object at 0x7fb...,clothes-woman_jacket,10783,2023-02-21,2023-02-21
3,126,clothes-child_clothes,medium_conditionbad_condition,[<shapely.geometry.point.Point object at 0x7fb...,13127,avaliable,697,730,81,[<shapely.geometry.point.Point object at 0x7fb...,clothes-woman_jacket,12623,2023-02-22,2023-02-24
4,739,clothes-woman_clothes,good_condition,[<shapely.geometry.point.Point object at 0x7fb...,13435,not_available,409,717,572,[<shapely.geometry.point.Point object at 0x7fb...,appliance-sandwich_maker,14195,2023-02-20,2023-02-22


In [15]:
mock_data['item_address'] = toilette_data['Street']
mock_data['item_lat'] = toilette_data['Latitude']
mock_data['item_lng'] = toilette_data['Longitude']
mock_data['item_postcode'] = toilette_data['PostalCode']


In [16]:
mock_data.head()

,item_id,item_name-item_category-item,item_condition,item_lat_lng,item_postcode,item_status,userwhoposted_id,userwhopickedup_id,userwhochangeditemstatus_id,userwhochangeditemstatus_lat_lng,searched_item_name-searched_item_category-searched_item,searched_postcode,item_datetime_posted,item_datetimechangeditemstatus,item_address,item_lat,item_lng
0,485,furniture-rollcontainers,medium_conditionbad_condition,[<shapely.geometry.point.Point object at 0x7fb...,10178.0,not_available,246,188,922,[<shapely.geometry.point.Point object at 0x7fb...,miscelaneaous-bicycle,13347,2023-02-23,2023-02-25,Alexanderplatz 1 (10-22 Uhr),52.521463,13.412954
1,584,miscelaneaous-ski_skates,good_condition,[<shapely.geometry.point.Point object at 0x7fb...,13597.0,not_available,900,256,652,[<shapely.geometry.point.Point object at 0x7fb...,furniture-cabinet,12103,2023-02-23,2023-02-23,"Spandauer Markt , Markt 5 (10-18 Uhr)",52.536861,13.205028
2,953,appliance-washing_machine,good_condition,[<shapely.geometry.point.Point object at 0x7fb...,14199.0,not_available,178,326,512,[<shapely.geometry.point.Point object at 0x7fb...,clothes-woman_jacket,10783,2023-02-21,2023-02-21,Hohenzollerndamm (0-24 Uhr),52.476679,13.281890
3,126,clothes-child_clothes,medium_conditionbad_condition,[<shapely.geometry.point.Point object at 0x7fb...,13597.0,avaliable,697,730,81,[<shapely.geometry.point.Point object at 0x7fb...,clothes-woman_jacket,12623,2023-02-22,2023-02-24,"Rathaus Spandau, Altstädter Ring (0-24 Uhr)",52.535470,13.200429
4,739,clothes-woman_clothes,good_condition,[<shapely.geometry.point.Point object at 0x7fb...,12163.0,not_available,409,717,572,[<shapely.geometry.point.Point object at 0x7fb...,appliance-sandwich_maker,14195,2023-02-20,2023-02-22,Schloßstr. ggü. 33-35 (0-24 Uhr),52.457270,13.321433


In [17]:
mock_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356 entries, 0 to 355
Data columns (total 17 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   item_id                                                  356 non-null    int64  
 1   item_name-item_category-item                             356 non-null    object 
 2   item_condition                                           356 non-null    object 
 3   item_lat_lng                                             356 non-null    object 
 4   item_postcode                                            327 non-null    float64
 5   item_status                                              356 non-null    object 
 6   userwhoposted_id                                         356 non-null    int64  
 7   userwhopickedup_id                                       356 non-null    int64  
 8   userwhochangeditemstatus_id   

## Exploring distances between item and user

**Method 1 - `Using math`**

Method 2 - Using `shapely.geometry`, `pyproj` and `math` **[PLAN FOR AN UPDATE]**

**[CONTINUE FROM HERE]** I used the toilete dataset only to test.

- change the dataset to the mock
- adapt variables

In [18]:
data = toilette_data
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356 entries, 0 to 447
Data columns (total 27 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   LavatoryID                        356 non-null    object 
 1   Description                       356 non-null    object 
 2   City                              356 non-null    object 
 3   Street                            356 non-null    object 
 4   Number                            0 non-null      float64
 5   PostalCode                        356 non-null    int64  
 6   Country                           356 non-null    object 
 7   Longitude                         356 non-null    float64
 8   Latitude                          356 non-null    float64
 9   isOperatedByWall                  356 non-null    int64  
 10  isHandicappedAccessible           356 non-null    int64  
 11  Price                             356 non-null    float64
 12  canBePay

In [19]:
import numpy as np

# # Fake user data
# user_coordinates=[52.52646152725616, 13.376623742933933]
# lat1 = np.radians(52.52646152725616)
# lon1 = np.radians(13.376623742933933)

user_coordinates=[52.482288, 13.419831] #id user 348
lat1 = np.radians(52.482288) #id item 348
lon1 = np.radians(13.419831) #id item 348

item_coordinates=[52.482288, 13.419831] #id item 468
item_lat = np.radians(52.513802) #id item 468
item_lon = np.radians(13.322393) #id item 468

In [20]:
from math import sin, cos, sqrt, atan2, radians

# approximate radius of earth in km
R = 6373.0
data['lat2'] = np.radians(data['Latitude'])
data['lon2'] = np.radians(data['Longitude'])

data['dlon'] = data['lon2'] - lon1
data['dlat'] = data['lat2'] - lat1

data['a'] = np.sin(data['dlat'] / 2)**2 + cos(lat1) * np.cos(data['lat2']) * np.sin(data['dlon'] / 2)**2
data['c'] = 2 * np.arctan2(np.sqrt(data['a'] ), np.sqrt(1 - data['a'] ))

data['geo_distance'] = R * data['c']

data['geo_distance']

0       4.382237
1      15.757762
2       9.365493
3      15.987832
4       7.224813
         ...    
442    19.551439
444    18.779189
445    13.397002
446    15.794684
447     9.400179
Name: geo_distance, Length: 356, dtype: float64

In [21]:
data['geo_distance'].nsmallest(5)

228    0.302305
113    0.543396
119    0.638492
15     0.700278
222    1.055813
Name: geo_distance, dtype: float64

In [22]:
data.head()

,LavatoryID,Description,City,Street,Number,PostalCode,Country,Longitude,Latitude,isOperatedByWall,...,neu in Phase 2 (ab 2/23),Price_new,Price_new.1,lat2,lon2,dlon,dlat,a,c,geo_distance
0,Wall_5339,Toilette Alexanderplatz,Berlin,Alexanderplatz 1 (10-22 Uhr),NaN,10178,Deutschland,13.412954,52.521463,1,...,0.0,0.5,0.5,0.916672,0.234100,-0.000120,0.000684,1.182072e-07,0.000688,4.382237
1,Wall_5348,Toilette Breite Str.,Berlin,"Spandauer Markt , Markt 5 (10-18 Uhr)",NaN,13597,Deutschland,13.205028,52.536861,1,...,0.0,0.5,0.5,0.916941,0.230471,-0.003749,0.000952,1.528414e-06,0.002473,15.757762
2,Wall_120224,"Berliner Toilette, Hohenzollerndamm",Berlin,Hohenzollerndamm (0-24 Uhr),NaN,14199,Deutschland,13.281890,52.476679,1,...,0.0,0.0,0.0,0.915891,0.231813,-0.002408,-0.000098,5.399000e-07,0.001470,9.365493
3,Wall_120941,"Berliner Toilette, Rathaus Spandau, Altstädter...",Berlin,"Rathaus Spandau, Altstädter Ring (0-24 Uhr)",NaN,13597,Deutschland,13.200429,52.535470,1,...,0.0,0.0,0.0,0.916917,0.230391,-0.003829,0.000928,1.573371e-06,0.002509,15.987832
4,Wall_120494,"Berliner Toilette, Schloßstr.",Berlin,Schloßstr. ggü. 33-35 (0-24 Uhr),NaN,12163,Deutschland,13.321433,52.457270,1,...,0.0,0.0,0.0,0.915552,0.232503,-0.001717,-0.000437,3.212961e-07,0.001134,7.224813
